In [35]:
'''
To change the theme use the following command:
!jt -t <theme-name>

Theme names: 
1) chesterish (currently using)
2) onedork
3) monokai
 more here: https://github.com/dunovank/jupyter-themes
 
 
 other: https://towardsdatascience.com/bringing-the-best-out-of-jupyter-notebooks-for-data-science-f0871519ca29
'''

'\nTo change the theme use the following command:\n!jt -t <theme-name>\n\nTheme names: \n1) chesterish\n2) onedork\n3) monokai\n more here: https://github.com/dunovank/jupyter-themes\n'

Now that we have gathered all the data, its time to clean and prepare them for our clustering algorithm

In [1]:
import pandas as pd
import progress_bar

In [391]:
df_all_news = pd.read_excel("data\\original_data\\all_news.xlsx")
df_all_news.drop(['Unnamed: 0'], axis=1, inplace=True) #drop the unecessary columns
initial_size = len(df_all_news)#store the number of records. Will be used for statistics

In [392]:
#count the number of records for each category and store it in a new dataframe
df_process_stats = df_all_news.groupby(['Category'], sort=False).size().reset_index(name='Count_Pre_Processing')

#drop all the blank cells
df_all_news.dropna(subset=['Title', 'Text'], inplace=True) 
#drop all the duplicate cells
df_all_news.drop_duplicates(subset=['Title', 'Text'],keep=False, inplace=True) 

#count the number of records for each category after the empty & duplicated cells removal
df_process_stats['Count_Post_Processing']=df_all_news.groupby(['Category'], sort=False).size().reset_index(name='Count_Post_Processing')['Count_Post_Processing']

In [393]:
def difference_percentage(num_A, num_B):
    if (num_A >= num_B):
        temp = ((num_B-num_A)/num_A) * 100
        temp = round(temp, 2) #keep only two digits after decimal
    else:
        temp = ((num_A-num_B)/num_B) * 100
        temp = round(temp, 2) #keep only two digits after decimal
    return temp

In [394]:
lossPercentage = []
for index, row in df_process_stats.iterrows():
    temp = difference_percentage(row['Count_Pre_Processing'], row['Count_Post_Processing'])
    lossPercentage.append(str(temp)+"%")
    
df_process_stats['Records_Loss']=lossPercentage

In [395]:
#TO IMPROVE THIS MAYBE MAKE SOME VISUALISATIONS?
total_loss = round((((len(df_all_news)-initial_size)/initial_size)*100), 2)
print("Total records pre-processing: " + str(df_process_stats['Count_Pre_Processing'].sum()))
print("Total subtracted records: " + str(df_process_stats['Count_Post_Processing'].sum()-df_process_stats['Count_Pre_Processing'].sum()))
print("Total records post-processing: " + str(df_process_stats['Count_Post_Processing'].sum()))

print("Total loss: " + str(total_loss) +"% of the initial records")
df_process_stats

Total records pre-processing: 493432
Total subtracted records: -155554
Total records post-processing: 337878
Total loss: -31.52% of the initial records


,Category,Count_Pre_Processing,Count_Post_Processing,Records_Loss
0,entertainment,50282,34150,-32.08%
1,financial,47851,44036,-7.97%
2,political,87157,37076,-57.46%
3,sport,156899,81799,-47.87%
4,technology,41476,37810,-8.84%
5,travel,49470,46520,-5.96%
6,world,60297,56487,-6.32%


In [396]:
#delete future unused variables
del index, row, temp, initial_size, lossPercentage, df_process_stats, total_loss

In [397]:
#df_all_news['Title'].duplicated().any()
#df_all_news[ids.isin(ids[ids.duplicated()])].sort("Title")
len(df_all_news)

337878

In [398]:
df_all_news[df_all_news.duplicated(['Text'], keep=False)]

,Title,Text,Category
14,Even Gina Rodriguez Doesnt Know What the Heck Happened at the End of Jane the Virgins Season 2 Premiere,"""Shaandaar"" alongside Alia Clinton 'shouldn't",entertainment
15,Box Office Top 20 Pan flops with $15.3 million,"""Shaandaar"" alongside Alia Clinton 'shouldn't",entertainment
46,Club Disney to premiere in new Hollywood Studios venue,,entertainment
52,You Probably Have a Lot of Questions After The Flashs Mother of a Reveal So Here Are Some Answers,,entertainment
53,Pitchfork Acquired By Conde Nast,,entertainment
...,...,...,...
493288,"Teen charged in deaths of dad, dad's fiancee and their son found in burned California cabin",,world
493328,Form 8.3 - Ball UK Acquisition Limited (Rexam plc),( Source : Rexam plc) FORM 8.3 PUBLIC OPENING POSITION DISCLOSURE/DEALING DISCLOSURE BY A PERSON WITH INTERESTS IN RELEVANT SECURITIES REPRESENTING 1% OR MORE Rule 8.3 of the Takeover Code (the ' Code') 1 . KEY INFORMATION (a) Full name of,world
493361,Tata Power is awarded the prestigious Porter Prize for Excellence in Corporate Integration & Governance ...,,world
493365,Pro-Houthi women protest against airstrikes,,world


### Count the number of words of each article - Time comparisons

In [409]:
t1 = time.time() 

df_all_news['Text_TotalWords'] = df_all_news['Text'].str.split().str.len()

t2 = time.time() 
print("Process completed.\nTime taken:") 
print("{:.2f}".format(round(t2-t1, 2))+" seconds.")


Time taken:
81.24075174331665


In [414]:
t1 = time.time() 
#df.col.apply(lambda x: len(x.split()))
df_all_news['Text_TotalWords'] = df_all_news.Text.apply(lambda x: len(str(x).split()))

t2 = time.time() 
print("Process completed.\nTime taken:") 
print("{:.2f}".format(round(t2-t1, 2))+" seconds.")


Process completed.
Time taken:
10.50 seconds.


In [416]:
df_all_news.reset_index(drop=True, inplace=True)
df_all_news.head(3)

,Title,Text,Category,Text_TotalWords
0,Tracy Morgan performs first stand-up show since accident,"Tracy Morgan performs first stand-up show since accident Tracy Morgan is coming back with a bang. Post to Facebook Tracy Morgan performs first stand-up show since accident Tracy Morgan is coming back with a bang. Check out this story on wtsp.com: http://on.wtsp.com/1REw1aj Cancel Send A link has been sent to your friend's email address. Posted! A link has been posted to your Facebook feed. Join the Conversation To find out more about Facebook commenting please read the Conversation Guidelines and FAQs Tracy Morgan performs first stand-up show since accident USA Today, USAToday.com 9:02 a.m. EDT October 13, 2015 Actor Tracy Morgan arrives at The 67th Annual Primetime Emmy Awards Governors Ball at the Los Angeles Convention Center on September 20, 2015 in Los Angeles, California. (Photo: Kevin Winter/Getty Images) CONNECT TWEET LINKEDIN COMMENT EMAIL MORE \nTracy Morgan is coming back with a bang. \nThe comedian, who has been recovering from a traumatic brain injury from a 2014 car a...",entertainment,380
1,President Barack Obama interviews author Marilynne Robinson,"FCC Information President Barack Obama interviews author Marilynne Robinson (AP Photo/Susan Walsh, File). FILE - In this Oct. 8, 2015, file photo, President Barack Obama speaks at the Congressional Hispanic Caucus Instituteβ€™s (CHCI) 38th Anniversary awards gala in Washington. Hillary Rodham Clinton's use of a private email ser... \nNEW YORK (AP) - For this interview, President Barack Obama was the one asking the questions. \nIn a discussion held last month in Iowa and posted Monday by The New York Review of Books, Obama asked Pulitzer Prize-winning author Marilynne Robinson about faith, democracy, education and writing. Obama explained that he met with Robinson because he was anxious to have an unscripted discussion with someone he admired. \n""One of the things that I don't get a chance to do as often as I'd like is just to have a conversation with somebody who I enjoy and I'm interested in,"" he told Robinson, ""to hear from them and have a conversation with them about some of the...",entertainment,333
2,"I will perform at night clubs, shows β€“ Pastor Ofori Amponsah","Source: Ghana | Hitz FM | Euphemia Akpalu Date: 12-10-2015 Time: 02:10:52:pm Ofori Amponsah \nHighlife-musician-turned-pastor, Ofori Amponsah, says he will perform at nightclubs and even at alcoholic company-sponsored events if contracted to. \nβ€People don't actually know why I will do it, but I will do it and it's a secret I don't want to reveal,β€ he told a fan. \nHe added that, β€I cannot judge because there are no laws that forbid the sale of alcohol, I will perform but [I'll] not be a brand ambassador for alcoholic beverages.β€ \nHe said this while promoting his new single 'Alewa' which features Sarkodie. He officially announced his return to highlife music in an interview on Hitz FM's Showbiz Fylla with Amanda Jissih. \nOfori Amponsah went into hibernation in 2008 - after releasing his last major highlife song - to concentrate on Godβ€™s work and to record gospel music. \nThe musician, who recently announced his return to doing highlife music even though he is the Head Pasto...",entertainment,404


In [429]:
df_all_news_processed = df_all_news.copy()
df_all_news_processed = df_all_news_processed[df_all_news_processed['Text_TotalWords'] > 10]  

In [430]:
#pd.options.display.max_colwidth = 50
#pd.options.display.max_colwidth = 1000
df_all_news_processed.loc[df_all_news_processed['Category'] == 'travel']

,Title,Text,Category,Text_TotalWords
234871,Californiaβ€™s North County rewards cyclists with stunning views,"Californiaβ€™s North County rewards cyclists with stunning views Comment Stuttgart Daily Leader - Stuttgart, AR By Alan Snel Posted Jul. 28, 2015 at 11:04 AM Updated at 11:05 AM Posted Jul. 28, 2015 at 11:04 AM Updated at 11:05 AM The feeling had been nagging at me for years, the urge to ride a bicycle along the Pacific Ocean in North County, that area north of San Diego. From Oceanside through Carlsbad and Encinitas to Solana Beach, Del Mar and Torrey Pines State Natural Reserve. All on old U.S. 101, the coastal road. South of the super sprawl of Los Angeles is that strip of roadway along the Pacific Ocean that has the best views and craft beer and bike riding youΒ’??ll find anywhere. On a day in May, I biked from Oceanside south to La Jolla, about 25 miles or so, and was returning north when I caught up with a bicyclist in Del Mar named Scott, a fertility drug researcher for a pharmaceutical company outside San Diego. As bicyclists do, we chatted and pedaled in rhythm as we cycle...",travel,742
234872,Asian stocks hit 3-week lows as China gloom spreads,"Other Articles China stocks fall as Beijing renews support, Hong Kong up Gold prices stuck near 5-1/2-year low as Fed rate hike looms Stocks that are likely to be in focus today: Maruti Suzuki, Tech Mahindra and more Asian stocks fell to three-week lows on Tuesday as a deepening rout in Chinese stocks erased risk appetite β€“ sending investors flocking to safe-haven instruments such as government bonds and the Japanese yen.\nMSCIβ€™s broadest index of Asia-Pacific shares outside Japan fell 0.8 percent in early deals, its lowest level since July 9 as mainland Chinese indexes opened 2- 5 percent lower.\nβ€Volatility is the enemy of investor appetite,β€ said the head of index trading at a U.S. fund. β€Any sign of government support to prop up the market will be used by investors to exit the market completely rather than add fresh positions.β€\nSince hitting a peak in early June, Chinese shares have gone through a roller-coaster ride with main China indexes falling by a third in less t...",travel,543
234873,Mexico struggles to clean up surge of seaweed washing onto beaches around Cancun resort,"MEXICO CITY - Authorities on Mexico's Caribbean coast said Monday they are redoubling efforts to remove tons of sargassum seaweed that has been washing ashore in recent weeks.\nThe state government of Quintana Roo, where the resort of Cancun is located, said there have been no reports of tourists cancelling visits because of the problem.\nGov. Roberto Borge said the seaweed removal efforts will focus on much of the coast, from Holbox in the north down past Tulum to the south.\nPhotos issued by the Cancun city government show piles of brown seaweed on some normally pristine white beaches. By last week, the city said it had raked or shovelled up 500 cubic meters of sargassum.\nBorge said the cause of the invasion is still unknown, though it could be due to high levels of nutrients in ocean water or changes in ocean temperatures, currents or wind patterns.\nAuthorities have to be careful, because there are two groups of visitors they don't want to disturb with overly aggressive remova...",travel,295
234874,Chinese airline launches Xiamen-Amsterdam route,"Chinese airline launches Xiamen-Amsterdam route Adv: \nJul 28, 2015 Adv: \nXIAMEN, China and AMSTERDAM, Netherlands - On July 26, Xiamen Airlines officially launched the carrier's very first intercontinental route, the Xiamen-Amsterdam route. The new route marks a key milestone in the airline's implementation of its globalization strategy. \nXiamen Airlines will operate the new route, flight numbers MF811 and MF812, with a Boeing 787 Dreamliner featuring 4 seats in first class, 18 in business and 214 in economy. The flight departs Xiamen at 11:50 p.m. every Tuesday, Thursday and Sunday, and arrives at Amsterdam's Schiphol at 5:45 a.m.

In [448]:
df_all_news_processed.Text.replace('\n',' ', regex=True, inplace=True)

#remove non-Ascii chars
df_all_news_processed.Text.replace(r'[^\x00-\x7F]',' ', regex=True, inplace=True) 

In [450]:
df_all_news_processed.loc[df_all_news_processed['Category'] == 'world']

,Title,Text,Category,Text_TotalWords
281391,Who needs a limo? Pope Francis opts for a more modest Fiat,"Photo: Andrew Harnik, AP Image 1 of / 3 Caption Close Image 1 of 3 Pope Francis waves from a Fiat 500 as his motorcade departs from Andrews Air Force Base, Md., Tuesday, Sept. 22, 2015, where President and Mrs. Obama welcomed him. Photo: Andrew Harnik, AP Pope Francis waves from a Fiat 500 as his motorcade departs from... Image 2 of 3 Pope Francis arrives at the Apostolic Nunciature, the Vatican's diplomatic mission in the heart of Washington, Tuesday, Sept. 22, 2015, in a Fiat 500. Pope Francis will visit the White House on Wednesday, becoming only the third pope to visit the White House. Photo: Cliff Owen, AP Pope Francis arrives at the Apostolic Nunciature, the Vatican's... Image 3 of 3 Pope Francis waves from a Fiat 500L as his motorcade departs Andrews Air Force Base, Md., Tuesday, Sept. 22, 2015, after being greeted by President Barack Obama and first lady Michelle Obama. Photo: Andrew Harnik, AP Pope Francis waves from a Fiat 500L as his motorcade departs... Who needs a limo...",world,560
281392,2 Canadians among 4 abducted on Philippine island,"The Philippine military says unidentified gunmen have abducted two Canadians, a Norwegian resort manager and a Filipino woman from a southern Philippine island....",world,23
281393,Clinton proposes prescription drug cap: Will it rally cash-strapped voters?,"Democratic presidential candidate Hillary Clinton will try to rally voters Tuesday with a plan to build on President Obama's reforms for health insurance by altering the way drug companies do business. Mrs. Clinton plans to tell voters in Iowa about her plan for a $250 monthly cap on out-of-pocket costs for prescription drugs. She wants the government to stop allowing drug companies to deduct what they spend to market new drugs. The marketing of new drugs directly to consumers has increased as the former practice of advertising directly to prescribing doctors has become more tightly regulated in recent years. ""It is time to deal with sky-rocketing out-of-pocket costs,"" Clinton said Monday during a campaign stop in Little Rock, Ark., according to Reuters. Recommended: How much do you know about Hillary Rodham Clinton? Take our quiz. Clinton's announcement comes just after the pharmaceutical company Turing Pharmaceuticals raised the price of Daraprim by 5,000 percent, reports the BBC...",world,518
281394,Suicide car bomber targets Somalia's presidential palace,"Updated 5 hours, 6 minutes ago NAIROBI, Kenya (AP) A Somali police official says a suicide car bomber detonated an explosives-laden vehicle at the gate of the Somalia's presidential palace in the capital on Monday evening, killing at least two soldiers. Capt. Mohamed Hussein said the bomber rammed his vehicle near soldiers guarding the compound in Mogadishu that is the official residence of the president, prime minister and the speaker of Parliament. Hussein said at least two soldiers died in the blast and that the death toll could rise. There was no immediate claim of responsibility for the attack, but the Somali Islamic extremist group al-Shabab often carries out such attacks.",world,112
281395,Why Syrian refugees donβ€™t flee to Islamic State caliphate,"People around the world have reacted differently to images of refugees desperately trying to reach Europe from Syria s four-year civil war. Yet one group, Islamic State (IS), may be the most unsettled. Since last year, the jihadi group has tried to set up an idealized Islamic society, or caliphate, and attract Sunni Muslims with its anti-Western message. Instead, hundreds of thousands of Syrian Muslims are now choosing to flee to Western countries. IS has lost one of its strongest recruitment lines that the West is no place to live or emulate. This is only the latest sign that IS contains the seeds of its own demise. Many of the group s internal woes have come to light in recent months, mainly from disenchanted fig

In [341]:
'''
0) 
    *replace \n with space (DONE)
    *remove non-ascii chars (DONE)
    *remove single chars and multiple spaces
    *remove stopwords and multiple spaces
1) split the words and find the most frequently words. Bi-grams, tri-grams Decide if they are part of the article or not
and then eliminate them in order to reduce the noise. To do that put them in a list with wordsToDeleteArray

2) for sure delete 'Updated: 12:30 a.m.'' or similar time 
    *  Updated: 6:48 6:48
    *  (Photo: KING-TV)
    *  Photo: Andrew Harnik
    *  Posted:
    *  ( Source : National Constitution Center )
    *  09/30/2015 - 4:02 pm
3) (After) regex to remove the times such as 15:20 a.m.
4) We may keep: 
    * the numbers because of the sports category
    * Minus and percentages -2.48 % (financial and world)
    * dots
5) replace the selected symbols with space
6) remove multiple spaces
7) remove stopwords
8) remove chars that are single such as "s"/. If you replace the symbol "That's" then it's left "That s"
9) remove dates?
10) remove links
'''

'\n0) replace \n with space\n1) split the words and find the most frequently words. Decide if they are part of the article or not\nand then eliminate them in order to reduce the noise. To do that put them in a list with wordsToDeleteArray\n\n2) for sure delete \'Updated: 12:30 a.m.\'\' or similar time  / \'Photo: \'?\n3) (After) regex to remove the times such as 15:20 a.m.\n4) We may keep: the numbers because of the sports category\n5) replace the selected symbols with space\n6) remove multiple spaces\n7) remove stopwords\n8) remove chars that are single such as "s"/. If you replace the symbol "That\'s" then it\'s left "That s"\n'